In [ ]:
from .utils import *


import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

# magic word for producing visualizations in notebook
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')



In [ ]:
azdias = pd.read_csv('../data/azdias.csv', sep=';')
customers=pd.read_csv('../data/customers.csv',sep=';')




### 1.5 Cluster Analysis<a name="cluster"></a>

Now, unsupervised machine learning techniques will be used in order to identify hidden patterns in the data, clustering the population into different groups, each one composed of people with similar characteristics.

With the defined clusters, it will be possible to perform a new comparison between customers and the general population. The difference is that, this time, the comparison won't be performed over one dimension (one variable), but over the different groups created through the effect that all the variables together have on these groups.

#### 1.5.1 Feature Engineering<a name="feateng2"></a>

First, there will be one more feature engineering process, this time over `CAMEO_DEU_2015` feature. Since there are 44 different classifications, they will be grouped according to the behavior presented when comparing customers and the general population, following this code:
* `0`: *underrepresented classes among clients*;
* `1`: *almost equally represented with slight underrepresentation tendency*;
* `2`: *almost equally represented with slight overrepresentation tendency*;
* `3`: *overrepresented classes among clients*.


In [ ]:
# Applying transform_cameo_deu on azdias dataframe:
azdias = transform_cameo_deu(azdias)

Since new features were created during the process, a complementary dictionary will be created to specify the new columns dtypes:


#### 1.5.2 NaN Values<a name="nan"></a>

The approach that will be used to fill in *nan* values is the following:
* `Numerical` and `ordinal` variables will have *nan* values replaced with the median of the existing values;

* `Categorical` variables will be binarized (One-Hot Encoding), and *nan* values will be indirectly considered.

In [ ]:
# Defining function to replace nan values with the median or create dummy variables for following specific distribution:
def deal_with_nan_values(df, dict1 = dtypes_dict, dict2 = new_feat_dtypes_dict):
    '''
    It deals with nan values in two different ways:
    * if the feature is defined as 'num' (numerical), it fills nan values with the median value;
    * if the feature is defined as 'cat' (categorical) or 'bin' (binary), it creates dummy variables
      and deletes the original column.

    Inputs:
    df: original dataframe;
    dict1: main dictionary mapping columns and dtypes;
    dict2: auxiliar dictionary mapping new columns and dtypes.

    Output:
    df: dataframe without nan values.
    '''
    # Selecting columns with nan values:
    nan_cols = list(df.columns[df.isnull().sum() != 0])

    # For each column, replace nan values:
    for col in nan_cols:

        # Verify in which dictionary the column is:
        if col in list(dict1.keys()):
            dict_ = dict1
        else:
            dict_ = dict2

        # Verify dtype, if numeric:
        if dict_[col] == 'num':
            # Fill in nan values with median:
            df[col].fillna(np.nanmedian(df[col]), inplace = True)

        else:
            # Delete original column and cocatenate dummy columns:
            df = pd.concat([df.drop(col, axis = 1), pd.get_dummies(df[col], prefix = col, \
                                                                   prefix_sep = '_', dummy_na = False)], axis = 1)

    return df

#### 1.5.3 Standardizing Data<a name="standard"></a>

Even after the feature selection process, many columns were left to be analyzed. Because of that, Principal Component Analysis will be applied to the data.

In order to apply the `PCA` algorithm, the values need to be on the same scale. For that, one function will be defined to fit the model to the data, and another to use the fitted model to transform the data:

In [ ]:
# Fitting StandardScaler model:
def fit_std_scaler(df):
    '''
    It uses the dataframe to fit the StandardScaler model.

    Input:
    df: dataframe used to fit the model.

    Output:
    cols: list of columns used in the fitting process;
    std_model: fitted StandardScaler model.
    '''
    # Instantiating scaler:
    std_scal = StandardScaler()

    # Columns used:
    cols = list(df.columns)

    # Fitting the model:
    std_scal.fit(df)

    return cols, std_scal

In [ ]:
# Fitting StandardScaler model:
def transform_std_scaler(df, cols, scaler):
    '''
    It uses the fitted StandardScaler model to transform the data.

    Input:
    df: dataframe used to be transformed;
    cols: list of columns to assure dataframe compatibility.
    scaler: fitted StandardScaler model.

    Output:
    df: transformed dataframe.
    '''
    # Dataframe columns compatibility:
    for col in cols:
        if col not in df.columns:
            df[col] = 0

    # Scaling data:
    df = pd.DataFrame(scaler.transform(df), columns = df.columns)

    return df

Since there are different levels of information, according to the informational spreadsheet, the dimensionality reduction, in this case, the `PCA` technique, will not be applied to the whole dataset at once. Different components will be created for different levels of information.

Because of that, the dataset will be split according to their information level, and the transformations will be applied to these subsets.


Next, all the transformations will be joined together in one function, and then it will be applied to the different levels of information.

In [ ]:
# Defining function the apply nan and standard scaler transformations:
def transform_data(df, info_level, dic = info_level, cols = None, scaler = None):
    '''
    It applies the functions that treat nan values and standardize the values.

    Inputs:
    df: original dataframe;
    info_level: string indicating which information level will be treated;
    dic: dictionary indicating the information level of each feature.

    Output:
    df: transformed dataframe with the level of information columns.
    '''
    # Selecting information level columns:
    sel_cols = dic[info_level]
    df = df[sel_cols]

    # Applying deal_with_nan_values function:
    df = deal_with_nan_values(df)

    # If scaler is not defined, create scaler:
    if scaler == None:
        # Fitting StandardScaler model:
        cols, scaler = fit_std_scaler(df)

        # Standardizing data:
        df = transform_std_scaler(df, cols, scaler)

        return df, cols, scaler

    else:
        # Standardizing data:
        df = transform_std_scaler(df, cols, scaler)

        return df

In [ ]:
# Applying data transformation on:
# 'person' info level:
azdias_pers, pers_cols, pers_scaler = transform_data(azdias, info_level = 'person')
azdias_pers.head()

In [ ]:
azdias_pers.shape

In [ ]:
# 'household' info level:
azdias_hh, hh_cols, hh_scaler = transform_data(azdias, info_level = 'household')
azdias_hh.head()

In [ ]:
azdias_hh.shape

In [ ]:
# 'microcell' info level:
azdias_mic, mic_cols, mic_scaler = transform_data(azdias, info_level = 'microcell')
azdias_mic.head()

In [ ]:
azdias_mic.shape

In [ ]:
# 'macrocell' info level:
azdias_mac, mac_cols, mac_scaler = transform_data(azdias, info_level = 'macrocell')
azdias_mac.head()

In [ ]:
azdias_mac.shape

In [ ]:
# 'community' info level:
azdias_com, com_cols, com_scaler = transform_data(azdias, info_level = 'community')
azdias_com.head()

In [ ]:
azdias_com.shape

In [ ]:
print('Information Level Summary:')
print('--------------------------')
print('Personal level has {} features.'.format(azdias_pers.shape[1]))
print('Household level has {} features.'.format(azdias_hh.shape[1]))
print('Microcell level has {} features.'.format(azdias_mic.shape[1]))
print('Macrocell level has {} features.'.format(azdias_mac.shape[1]))
print('Community level has {} features.'.format(azdias_com.shape[1]))

Considering the number of features after the transformations, `PCA` will be applied only on the first four levels of information:
* Personal;
* Household;
* Microcell;
* Macrocell.

The Community level has only three features, and because of that, they will be kept without further transformations.

#### 1.5.4 Dimensionality Reduction<a name="pca"></a>

Once the data is treated and standardized, before actually applying the dimensionality reduction, `PCA` will be applied with standard parameters in order to decide the number of components to keep.

To make the number of components decision, a *scree plot* will be created for each level of information data.

In [ ]:
# Defining function to create scree plot to decide how many components to keep:
def create_scree_plot(pca):
    '''
    It creates a scree plot for the fitted principal component analysis in order to define the number
    of components to keep.

    Input:
    pca: principal component analysis object fitted to data.
    '''
    # Getting the variance's percentage explained by each component:
    perc_vars = pca.explained_variance_ratio_

    # Getting cumulative values of the variance's percentage:
    cum_perc_vars = np.cumsum(perc_vars)

    # Defining the number of components:
    n_comp = len(pca.explained_variance_ratio_)

    # Defining the index for each component:
    index = np.arange(n_comp)

    # Creating scree plot:
    sns.set_theme(style = "whitegrid", font_scale = 1.1)
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (13, 5))

    # Drawing lineplot:
    sns.lineplot(x = index, y = cum_perc_vars,
                 markers = True, color = 'black').set(xlabel = 'Number of Principal Components',
                                                      ylabel = 'Explained Variance',
                                                      title = "Scree Plot - Cumulative Explained Variance",
                                                      ylim = (0,1.02))
    sns.despine()
    fig.show()

The purpose is to retain most of the data variability, and the few components as possible, simplifying the resulted data.

Since it's a client segmentation problem, the assumption is that the `person` information level must be more important than the `household` information level, which is more important than the `microcell` and the `macrocell` information levels, being the last one the more general in this scale.

Because of that, when deciding the number of components to keep, more important levels will have the number of components necessary to explain about 60% of the variance, while less important levels will be allowed to have a lower explained variance rate.

In [ ]:
# PERSON level:
pca = PCA()
pers_pca = pca.fit(azdias_pers)

# Getting the scree plot:
create_scree_plot(pers_pca)

In order to keep about 60% of the explained variance, in the `person` level **30 components** will be kept.

In [ ]:
# HOUSEHOLD level:
hh_pca = pca.fit(azdias_hh)

# Getting the scree plot:
create_scree_plot(hh_pca)

In the `household` level, **20 components** explain over 50% of the data variance.

In [ ]:
# MICROCELL level:
mic_pca = pca.fit(azdias_mic)

# Getting the scree plot:
create_scree_plot(mic_pca)

In [ ]:
# MACROCELL level:
mac_pca = pca.fit(azdias_mac)

# Getting the scree plot:
create_scree_plot(mac_pca)

Because of the scale of importance assumed before, `microcell` and `macrocell` levels will be allowed to have their rate of explained variance between 40 and 50%, keeping **10 components** for each one of them.

<h4>Applying PCA</h4>

In [ ]:
#____________________________________________________________
# PERSON:

# Instantiating pca object:
pca_pers = PCA(n_components = 30, random_state = 101)

# Fitting pca and applyting transformation on azdias dataset:
azdias_pers_pca = pca_pers.fit_transform(azdias_pers)

#____________________________________________________________
# HOUSEHOLD:

# Instantiating pca object:
pca_hh = PCA(n_components = 20, random_state = 201)

# Fitting pca and applyting transformation on azdias dataset:
azdias_hh_pca = pca_hh.fit_transform(azdias_hh)

#____________________________________________________________
# MICROCELL:

# Instantiating pca object:
pca_mic = PCA(n_components = 10, random_state = 301)

# Fitting pca and applyting transformation on azdias dataset:
azdias_mic_pca = pca_mic.fit_transform(azdias_mic)

#____________________________________________________________
# MACROCELL:

# Instantiating pca object:
pca_mac = PCA(n_components = 10, random_state = 401)

# Fitting pca and applyting transformation on azdias dataset:
azdias_mac_pca = pca_mac.fit_transform(azdias_mac)

In [ ]:
# Column names for general dataframe:
person_cols = ['pers01', 'pers02', 'pers03', 'pers04', 'pers05', 'pers06', 'pers07', 'pers08', 'pers09', 'pers10', 'pers11',
               'pers12', 'pers13', 'pers14', 'pers15', 'pers16', 'pers17', 'pers18', 'pers19', 'pers20', 'pers21', 'pers22',
               'pers23', 'pers24', 'pers25', 'pers26', 'pers27', 'pers28', 'pers29', 'pers30']

household_cols = ['hh01', 'hh02', 'hh03', 'hh04', 'hh05', 'hh06', 'hh07', 'hh08', 'hh09', 'hh10', 'hh11', 'hh12', 'hh13',
                  'hh14', 'hh15', 'hh16', 'hh17', 'hh18', 'hh19', 'hh20']

microcell_cols = ['mic01', 'mic02', 'mic03', 'mic04', 'mic05', 'mic06', 'mic07', 'mic08', 'mic09', 'mic10']

macrocell_cols = ['mac01', 'mac02', 'mac03', 'mac04', 'mac05', 'mac06', 'mac07', 'mac08', 'mac09', 'mac10']

# Concatenating pca dataframes together, also including the 3 features related to the community information level:
gen_azdias = pd.concat([pd.DataFrame(azdias_pers_pca, columns = person_cols),
                        pd.DataFrame(azdias_hh_pca, columns = household_cols),
                        pd.DataFrame(azdias_mic_pca, columns = microcell_cols),
                        pd.DataFrame(azdias_mac_pca, columns = macrocell_cols),
                        azdias_com], \
                       axis = 1)

gen_azdias.head()

To better understand how the original features compose these components created through `PCA`, a function will be defined to return the most important features for each component:

In [ ]:
# Defining function to show most important features for each component:
def analyze_component(component, top = 3):
    '''
    It returns the most important features related to the component.

    Inputs:
    component: integer indicating the number of the component to be analyzed;
    top: integer indicating the number of top features to return;
    '''
    # Checking on to which level of informations the component relates to:
    # PERSON component:
    if component in range(30):
        print('\nPERSON Component:')
        print('-----------------')
        pca = pca_pers
        features = list(azdias_pers.columns)

    # HOUSEHOLD component:
    elif component in range(30, 50):
        print('\nHOUSEHOLD Component:')
        print('--------------------')
        pca = pca_hh
        features = list(azdias_hh.columns)
        component = component - 30

    # MICROCELL component:
    elif component in range(50, 60):
        print('\nMICROCELL Component:')
        print('--------------------')
        pca = pca_mic
        features = list(azdias_mic.columns)
        component = component - 50

    # MACROCELL component:
    elif component in range(60, 70):
        print('\nMACROCELL Component:')
        print('--------------------')
        pca = pca_mac
        features = list(azdias_mac.columns)
        component = component - 60

    else:
        return(print('Component out of range (0-69)'))


    # Extracting features' weight on the desired component:
    weights = list(pca.components_[component])

    # Getting the index of ordered weights:
    ordered_idx = np.argsort(weights)[::-1]

    # Getting top indexes:
    top_pos_idx = ordered_idx[:top]
    top_neg_idx = ordered_idx[-top:]

    # Printing top positive weights on the component:
    print('\nComponent {}'.format(component))
    print('------------')
    print('\n* Top {} Positive Weights:'.format(top))
    print('--------------------------')
    for i in top_pos_idx:
        print('{}: {:.3f}'.format(features[i], weights[i]))

    # Printing top negative weights on the component:
    print('\n* Top {} Negative Weights:'.format(top))
    print('--------------------------')
    for j in top_neg_idx[::-1]:
        print('{}: {:.3f}'.format(features[j], weights[j]))

In [ ]:
# Testing analyze_component:
analyze_component(0, top = 5)

Unfortunately, there was no explanation provided for the feature `CJT_TYP_6` (it's only known to be related to *customer journey typology*). However, as an example, the **Component 0** related to the **person** information level will be interpreted without this specific feature.

`ALTERSKATEGORIE_GROB` relates to age classification through prename analysis, where higher values represent higher ages, indicating that these components represent elder people. This aspect can be reinforced when analyzing the highest negative weight related to the `YOUTH_DECADE` component.

`YOUTH_DECADE` indicates in which decade the person lived his/her youth period. In other words, the lower the decade, the elder the person is. Basically, these two variables represent the same information, but their values are in opposite directions, confirming that this component represents elder people.

It can also be seen that `SEMIO_ERL` and `SEMIO_LUST` are important features that positively represent this component. `SEMIO_ERL` describes if the person is eventful oriented while `SEMIO_LUST`indicates if the person is sensual minded. Higher values indicate lower affinity with that specific characteristic (*1 - highest affinity*, *7 - lowest affinity*). It tells that this component represents elder people that are *not* eventful oriented, nor sensual minded.

On the oher hand, `SEMIO_TRADV` and `SEMIO_REL` represent important aspects in the opposite direction, indicating that the component represents people that are traditional-minded and religious. At the same time, they have a rational mind (`SEMIO_RAT`).

`RETOURTYP_BK_S_5.0` indicates that the return type of these people is classified as *determined minimal-returner*, and the `FINANZ_ALENGER` indicates a high correlation to the *investor* financial typology, maybe giving a hint that these people may also be associated with higher incomes.

#### 1.5.5 Defining the Number of Clusters<a name="nclusters"></a>

Before classifying the data in different clusters, it's necessary to find the optimal number of clusters. For that, the *Elbow Method* will be used:

In [ ]:
# Defining function to create Elbow Method Visualization
def apply_elbow_method(df = gen_azdias):
    '''
    It returns a visualization that shows the sum of squared distances of samples to their closest cluster
    center for each attempt of number of clusters.

    Input:
    df: dataframe on which cluster analysis will be performed.
    '''
    # Defining number of clusters attempts:
    n_cluster = range(2, 32, 2)

    # Creating inertia list:
    wcss = list()

    # Perform K-Means for each attempt and extract its 'inertia' (sum of quared distances to their closest cluster):
    for n in n_cluster:
        kmeans = KMeans(n_clusters = n)
        kmeans.fit(df)
        wcss.append(kmeans.inertia_)

    # Creating the visualization:
    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (13, 5))

    # Drawing lineplot:
    sns.lineplot(x = n_cluster, y = wcss,
                 markers = True, color = 'black').set(xlabel = 'Number of Clusters',
                                                      ylabel = 'Sum of Squared Distances',
                                                      title = "Elbow Method - Optimizing Number of Clusters Choice")
    sns.despine()
    fig.show()

In [ ]:
# Checking Elbow visualization:
apply_elbow_method()

Unfortunately, there's no clear elbow indicating which would be the best number of clusters to be chosen. However, it's possible to see that the most expressive change in the inclination happens between 15 and 20 clusters.

With that, the clustering process will be built considering **18 clusters**.

#### 1.5.6 Applying Transformations on Customer Data<a name="transcustomer"></a>

Before proceeding to the clustering process, data transformation will be applied to the customers data:

In [ ]:
# CAMEO_DEU feature engineering:
customers = transform_cameo_deu(customers)

In [ ]:
# Applying data transformation on:
# 'person' info level:
customers_pers = transform_data(customers, info_level = 'person', cols = pers_cols, scaler = pers_scaler)

In [ ]:
# 'household' info level:
customers_hh = transform_data(customers, info_level = 'household', cols = hh_cols, scaler = hh_scaler)

In [ ]:
# 'microcell' info level:
customers_mic = transform_data(customers, info_level = 'microcell', cols = mic_cols, scaler = mic_scaler)

In [ ]:
# 'macrocell' info level:
customers_mac = transform_data(customers, info_level = 'macrocell', cols = mac_cols, scaler = mac_scaler)

In [ ]:
# 'community' info level:
customers_com = transform_data(customers, info_level = 'community', cols = com_cols, scaler = com_scaler)

In [ ]:
# PCA transformation:

#_______________________________________________________
# PERSON:

# Applyting transformation on customers dataset:
customers_pers_pca = pca_pers.transform(customers_pers)

#_______________________________________________________
# HOUSEHOLD:

# Applyting transformation on customers dataset:
customers_hh_pca = pca_hh.transform(customers_hh)

#_______________________________________________________
# MICROCELL:

# Applyting transformation on customers dataset:
customers_mic_pca = pca_mic.transform(customers_mic)

#_______________________________________________________
# MACROCELL:

# Applyting transformation on customers dataset:
customers_mac_pca = pca_mac.transform(customers_mac)

In [ ]:
# Concatenating pca dataframes together, also including the 3 features related to the community information level:
gen_customers = pd.concat([pd.DataFrame(customers_pers_pca, columns = person_cols),
                           pd.DataFrame(customers_hh_pca, columns = household_cols),
                           pd.DataFrame(customers_mic_pca, columns = microcell_cols),
                           pd.DataFrame(customers_mac_pca, columns = macrocell_cols),
                           customers_com], \
                          axis = 1)

gen_customers.head()

#### 1.5.7 Clustering<a name="clustering"></a>

In [ ]:
# Training KMeans algorithm and making cluster predictions:
# Instatiating KMeans algorithm for n clusters:
kmeans = KMeans(n_clusters = 18, random_state = 101)

# Fitting to df1 data:
kmeans.fit(gen_azdias)

# Predicting clusters on df1:
azdias_clusters = kmeans.predict(gen_azdias)

# Predicting clusters on df2:
customers_clusters = kmeans.predict(gen_customers)

#### 1.5.8 Evaluating Clusters<a name="clustereval"></a>

Once each observation was assigned to its correspondent cluster, the job now is to check which clusters are proportionally more representative among clients than in the general population. The same for the clusters that happen more frequently in the general population in comparison to the customers' group.

With that, it will be possible to understand the different combinations of features that result in a person being more likely to become a client or the other way around.

In [ ]:
# Defining a function to create a comparison barplot:
def compare_cluster_occurance(customers = customers_clusters, general = azdias_clusters):
    '''
    It creates a bor plot comparing the percentages related to each one the clusters borh for customers and
    the general population.

    Inputs:
    customers: array containing the assigned cluster for each customer observation;
    general: rray containing the assigned cluster for each general population observation.
    '''
    # Counting cluster occurences in customer population:
    customer_cluster_occ = np.bincount(customers)

    # Counting cluster occurences in general population:
    gen_pop_cluster_occ = np.bincount(general)

    # Creating dataframe with cluster proportional counting:
    cluster_df = pd.DataFrame(columns = ['general_population', 'customers'])
    n_pop = np.sum(gen_pop_cluster_occ)
    n_cust = np.sum(customer_cluster_occ)
    cluster_df.general_population = [(count / n_pop) * 100 for count in gen_pop_cluster_occ]
    cluster_df.customers = [(count / n_cust) * 100 for count in customer_cluster_occ]
    cluster_df['Cluster'] = cluster_df.index.values

    # Transforming df to melted version:
    melted_df = pd.melt(cluster_df, id_vars = ['Cluster'] , value_vars = ['general_population', 'customers'], \
                        var_name = 'Group', value_name ='Percentage')

    palette = {'customers': 'darkcyan', 'general_population': 'springgreen'}

    fig, ax = plt.subplots(nrows = 1, ncols = 1, figsize = (16.5, 5))
    sns.barplot(x = 'Cluster', y = 'Percentage', hue = 'Group', data = melted_df, palette = palette, ax = ax).set(
        title = 'Comparison Between Customers and General Population - Clusters')
    sns.despine(left=True, top = True)
    fig.show()

    # Computing percentage differences:
    perc_diff = [((cluster_df.customers[i] - cluster_df.general_population[i]) / cluster_df.general_population[i]) * 100 \
                 for i in range(cluster_df.shape[0])]
    # Getting ordered index:
    ordered_idx = np.argsort(perc_diff)[::-1]

    # Printing top 5 customers' clusters with highest representativity in comparison to general population:
    print('Top 5 Overrepresented Clusters among Customers:')
    print('-----------------------------------------------\n')
    for i in ordered_idx[:5]:
        print('Cluster {}: increased {:.1f} %'.format(i, perc_diff[i]))

    # Printing top 5 customers' clusters with lowest representativity in comparison to general population:
    print('\nTop 5 Underrepresented Clusters among Customers:')
    print('-----------------------------------------------\n')
    for i in ordered_idx[-5:][::-1]:
        print('Cluster {}: decreased {:.1f} %'.format(i, perc_diff[i]))

In [ ]:
compare_cluster_occurance()

To better understand what these clusters represent, `cluster center` will provide the most important components related to that specific cluster, and then it will be possible to go back to those specific components to understand the features that better represent them.

That way, it will be possible to develop a cluster overview.

In [ ]:
# Creating a function that, given one specific cluster, returns the most important features associated with it:
def cluster_most_important_features(cluster, kmeans = kmeans, top_comp = 3):
    '''
    Given a cluster, it searches for the most important components, and through them, search for
    the most important features related to those components.

    Input:
    cluster: integer indicating the target cluster (0-17);
    kmeans: trained kmeans model
    top_com: number of most important components to be analyzed.
    '''
    # Getting cluster center:
    cluster_centers = kmeans.cluster_centers_[cluster]

    # Getting more important components:
    pos_idx = np.argsort(cluster_centers)[::-1]
    pos_idx = pos_idx[:top_comp]

    # Analyzing components and extracting their most important features:
    print('Principal Components with Highest Impact on Cluster {}:'.format(cluster))
    print('--------------------------------------------------------')
    for comp in pos_idx:
        analyze_component(comp)

#### 1.5.8.1 Overrepresented Clusters<a name="over"></a>

**Cluster 2 - GREEN DREAMERS**

In [ ]:
# Analyzing Cluster 2:
cluster_most_important_features(2, top_comp = 3)

`CLUSTER 2` is the one with the highest overrepresentation, being proportionally over 205% more representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* their vacation habits can be classified as *nature fans*, already emphasizing the green tendency as an important aspect present in people represented by this cluster;

* the green aspect is reinforced with the *avant-garde* aspect, relating this cluster to people with the avant-garde mindset, which also relates them to the *green avant-garde* movements;

* they don't seem to be familiar minded;

* this cluster is representing mostly men that are not single, not critical-minded but are dreamers and social-minded.

**Cluster 9 - HIGH-SOCIETY TRADITIONAL ELDERS**

In [ ]:
# Analyzing Cluster 9:
cluster_most_important_features(9, top_comp = 3)

`CLUSTER 9` is also about 205% more representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* they are in advanced life stages, being represented as families or multiperson households, and also related to higher incomes;

* their social status can be classified as independents, house owners and top-earners;

* they are not dreamers, nor eventful oriented, but they can be represented as being critical and traditional-minded, as well as religious;

* the cluster describes elder people.

**Cluster 4 - EMPTY NEST, FULL WALLET**

In [ ]:
# Analyzing Cluster 4:
cluster_most_important_features(4, top_comp = 3)

`CLUSTER 4` is over 100% more representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* it represents mostly average to high-income couples;

* it could also represent house owner couples and top earner-couples of higher age;

* the third component brings aspects seen before, as being  traditional-minded, and religious, but not eventful oriented;

* it's possible to say that some aspects seen before are now applied mostly to couples with high-incomes from different ages, and also mature and top earner-couples.

#### 1.5.8.2 Underrepresented Clusters<a name="under"></a>

**Cluster 0 - LESS AFFORTUNATE BEGINNERS**

In [ ]:
# Analyzing Cluster 0:
cluster_most_important_features(0, top_comp = 3)

`CLUSTER 0` is 95% less representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* it represents people that could be classified as less affluent or poorer, mostly indicating pre-family couples and singles or young couples with children;

* their household incomes can be considered as lower or very low incomes, and their insurance typology is mostly classified as *social safety-driven*;

* in terms of health and shopping typologies, they could be described as *sanitary affine* and *demanding shoppers*;

* the microcell component indicates areas with a high share of cares built before 1994 that had two or more pre-owners, creating the picture of people with lower incomes that live in poorer areas.

**Cluster 16 - MULTI-GENERATION MONEY SAVERS**

In [ ]:
# Analyzing Cluster 9:
cluster_most_important_features(16, top_comp = 3)

`CLUSTER 9` is also about 95% less representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* this cluster describes people with low-income and average earners of higher age from multiperson households, and families classified as *two-generation household*;

* there is also a strong aspect of singles with low-incomes or average-earners families, and also average earners of younger age from multiperson households. This profile has a tendency of being related to people living in the Westside;

* their financial typology to be classified as *low financial interest* or *money-savers*.

**Cluster 14 - SECOND-HAND CAR CELLS**

In [ ]:
# Analyzing Cluster 14:
cluster_most_important_features(14, top_comp = 3)

`CLUSTER 14` is about 83% less representative in customers than in the general population.

Analyzing the most important components, that's how this cluster could be described:
* it represents people that could be classified as less affluent or poorer, mostly indicating pre-family couples and singles or young couples with children;

* the microcell component indicates areas with a high share of cares built before 1994 that had two or more pre-owners, creating the picture of people with lower incomes that live in poorer areas;

* it also describes areas with a high share of family houses, with a very low share of cars per household, and also close to city centers;

* most of these aspects were already highlighted in other clusters, but in this case, it seems to emphasize more the micro and macrocells aspects than the ones related to the person.

